In [1]:
%load_ext autoreload
%autoreload 2

## English Rest Semeval-2016 SB1 Slot3

In [2]:
%%writefile config.json
{
    "data_config": {
        "competition": "semeval",
        "domain": "rest",
        "language": "en",
        "test_filename": "/Volumes/My Passport/Datasets/Sentiment/ABSA16/ABSA16_Restaurants_En_Test.xml",
        "train_filename": "/Volumes/My Passport/Datasets/Sentiment/ABSA16/ABSA16_Restaurants_En_Train.xml"
    },
    "model_config": {
        "is_sequence_predictor": true,
        "char_dropout_p": 0.4,
        "char_embedding_dim": 4,
        "char_function_output_size": 30,
        "char_max_word_length": 30,
        "dense_size": 32,
        "dense_dropout": 0.4,
        "gram_dropout_p": 0.4,
        "gram_hidden_size": 32,
        "rnn_bidirectional": true,
        "rnn_dropout_p": 0.5,
        "rnn_hidden_size": 32,
        "rnn_output_dropout_p": 0.4,
        "rnn_n_layers": 2,
        "use_chars":  false,
        "use_crf": true,
        "use_pos": true,
        "use_word_embeddings": true,
        "word_embedding_dim": 300,
        "word_embedding_dropout_p": 0.4,
        "output_size": 25
    },
    "batch_size": 8,
    "embeddings_filename": "semeval-2016-en-rest-fasttext.txt",
    "epochs": 100,
    "lr": 0.001,
    "max_length": 650,
    "output_filename": "submission.xml",
    "patience": 2,
    "task_type": "12",
    "use_pretrained_embedding": true,
    "val_size": 0.2,
    "word_max_length": 30,
    "model_filename": "model.pt"
}

Overwriting config.json


In [3]:
from src.config import Config
from src.parser import get_data

config = Config()
config.load("config.json")
train_data, test_data = get_data(config.data_config)   

max_length = max(train_data.get_lengths() + test_data.get_lengths())
vocabulary = train_data.get_vocabulary().merge(test_data.get_vocabulary())
char_set = train_data.get_char_set()
print(vocabulary.size())
print(char_set)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Num of reviews: 350
Num of opinions: 2507
Max review length: 620
[<PosTaggedWord "Judging", VERB#VBG, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] at 0x11ba9f7f0>, <PosTaggedWord "from", ADP#IN, [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0] at 0x11ba9f828>, <PosTaggedWord "previous", ADJ#JJ, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Num of reviews: 90
Num of opinions: 859
Max review length: 438
[<PosTaggedWord "Yum", INTJ#UH, [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] at 0x11f0dc3c8>, <PosTaggedWord "!", PUNCT#., [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0] at 0x11f0dc240>]
[<PosTaggedWord "Yum", INTJ#UH, [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [4]:
from src.train import train_model
from src.config import get_targets_additionals

targets, additionals, rev_categories = get_targets_additionals(train_data)
model = train_model("config.json", train_data, vocabulary, char_set, targets, additionals)

{'AMBIENCE#GENERAL': 0, 'DRINKS#PRICES': 1, 'DRINKS#QUALITY': 2, 'DRINKS#STYLE_OPTIONS': 3, 'FOOD#PRICES': 4, 'FOOD#QUALITY': 5, 'FOOD#STYLE_OPTIONS': 6, 'LOCATION#GENERAL': 7, 'RESTAURANT#GENERAL': 8, 'RESTAURANT#MISCELLANEOUS': 9, 'RESTAURANT#PRICES': 10, 'SERVICE#GENERAL': 11}
{0: 'AMBIENCE#GENERAL', 1: 'DRINKS#PRICES', 2: 'DRINKS#QUALITY', 3: 'DRINKS#STYLE_OPTIONS', 4: 'FOOD#PRICES', 5: 'FOOD#QUALITY', 6: 'FOOD#STYLE_OPTIONS', 7: 'LOCATION#GENERAL', 8: 'RESTAURANT#GENERAL', 9: 'RESTAURANT#MISCELLANEOUS', 10: 'RESTAURANT#PRICES', 11: 'SERVICE#GENERAL'}
Use cuda:  False


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:58: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.start_transitions, -0.1, 0.1)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:59: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.end_transitions, -0.1, 0.1)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:60: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.transitions, -0.1, 0.1)


Unknown words in semeval-2016-en-rest-fasttext.txt: 218
Epoch: 0, train loss: 4201.781940569196, val loss: 3568.624959309896
Epoch: 1, train loss: 2959.0980730329243, val loss: 3188.3394775390625
Epoch: 2, train loss: 2820.4223214285716, val loss: 3104.1981065538193
Epoch: 3, train loss: 2732.054310825893, val loss: 2990.9011027018228
Epoch: 4, train loss: 2610.397096470424, val loss: 2867.8101976182725
Epoch: 5, train loss: 2506.9029227120536, val loss: 2778.753173828125
Epoch: 6, train loss: 2422.0680821010046, val loss: 2684.1905110677085
Epoch: 7, train loss: 2333.8517490931918, val loss: 2587.0965643988716
Epoch: 8, train loss: 2255.333035714286, val loss: 2496.976820203993
Epoch: 9, train loss: 2180.8762817382812, val loss: 2417.5761922200522
Epoch: 10, train loss: 2108.6323573521204, val loss: 2341.155280219184
Epoch: 11, train loss: 2045.3158639090402, val loss: 2269.6201375325522
Epoch: 12, train loss: 1979.3947745186942, val loss: 2196.188001844618
Epoch: 13, train loss: 1916

In [6]:
import torch
import numpy as np

predict("model.pt", "config.json", test_data, vocabulary, targets, additionals, rev_categories)
!head -n 1000 submission.xml

{'AMBIENCE#GENERAL': 0, 'DRINKS#PRICES': 1, 'DRINKS#QUALITY': 2, 'DRINKS#STYLE_OPTIONS': 3, 'FOOD#PRICES': 4, 'FOOD#QUALITY': 5, 'FOOD#STYLE_OPTIONS': 6, 'LOCATION#GENERAL': 7, 'RESTAURANT#GENERAL': 8, 'RESTAURANT#MISCELLANEOUS': 9, 'RESTAURANT#PRICES': 10, 'SERVICE#GENERAL': 11}
{0: 'AMBIENCE#GENERAL', 1: 'DRINKS#PRICES', 2: 'DRINKS#QUALITY', 3: 'DRINKS#STYLE_OPTIONS', 4: 'FOOD#PRICES', 5: 'FOOD#QUALITY', 6: 'FOOD#STYLE_OPTIONS', 7: 'LOCATION#GENERAL', 8: 'RESTAURANT#GENERAL', 9: 'RESTAURANT#MISCELLANEOUS', 10: 'RESTAURANT#PRICES', 11: 'SERVICE#GENERAL'}


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:58: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.start_transitions, -0.1, 0.1)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:59: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.end_transitions, -0.1, 0.1)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:60: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.transitions, -0.1, 0.1)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torchcrf/__init__.py:284: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  best_tags = [best_last_tag[0]]


<?xml version="1.0" ?>
<Reviews>
<Review rid="en_BlueRibbonSushi_478218171">
<sentences>
<sentence id="en_BlueRibbonSushi_478218171:0">
<text>Yum!</text>
<Opinions/></sentence>
<sentence id="en_BlueRibbonSushi_478218171:1">
<text>Serves really good sushi.</text>
<Opinions>
<Opinion target="sushi" category="FOOD#QUALITY" polarity="neutral" from="19" to="24"/>
</Opinions>
</sentence>
<sentence id="en_BlueRibbonSushi_478218171:2">
<text>Not the biggest portions but adequate.</text>
<Opinions>
<Opinion target="portions" category="FOOD#STYLE_OPTIONS" polarity="neutral" from="16" to="24"/>
</Opinions>
</sentence>
<sentence id="en_BlueRibbonSushi_478218171:3">
<text>Green Tea creme brulee is a must!</text>
<Opinions>
<Opinion target="Green Tea creme brulee" category="DRINKS#QUALITY" polarity="neutral" from="0" to="22"/>
</Opinions>
</sentence>
<sentence id="en_BlueRibbonSushi_478218171:4">
<text>Don't leave the restaurant without it.</text>
<Opinions/></sentence>
</sentences>
</Review>
<Revie

In [14]:
from src.embeddings import shrink_w2v
ALL_EMBEDDINGS_FILENAME = "/Volumes/My Passport/Models/Vectors/FastText/wiki.en.vec"
EMBEDDINGS_FILENAME = "semeval-2016-en-rest-fasttext.txt"
shrink_w2v(ALL_EMBEDDINGS_FILENAME, vocabulary, 10000, EMBEDDINGS_FILENAME)

Parsed words: 0, intersection: 0, unknown words:4091
Parsed words: 100000, intersection: 3487, unknown words:604
Parsed words: 200000, intersection: 3645, unknown words:446
Parsed words: 300000, intersection: 3706, unknown words:385
Parsed words: 400000, intersection: 3738, unknown words:353
Parsed words: 500000, intersection: 3760, unknown words:331
Parsed words: 600000, intersection: 3775, unknown words:316
Parsed words: 700000, intersection: 3782, unknown words:309
Parsed words: 800000, intersection: 3799, unknown words:292
Parsed words: 900000, intersection: 3809, unknown words:282
Parsed words: 1000000, intersection: 3818, unknown words:273
Parsed words: 1100000, intersection: 3826, unknown words:265
Parsed words: 1200000, intersection: 3832, unknown words:259
Parsed words: 1300000, intersection: 3835, unknown words:256
Parsed words: 1400000, intersection: 3840, unknown words:251
Parsed words: 1500000, intersection: 3844, unknown words:247
Parsed words: 1600000, intersection: 3851